In [ ]:
# Testing CARE implementation from CSBDeep library

In [ ]:
# !pip install tifffile
# !pip install sklearn
# !pip install scikit-image
!pip install import_ipynb
!pip install csbdeep

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 69 kB 4.4 MB/s 
     |████████████████████████████████| 2.9 MB 16.4 MB/s 
  Attempting uninstall: h5py
    Found existing installation: h5py 3.1.0
    Uninstalling h5py-3.1.0:
      Successfully uninstalled h5py-3.1.0


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
MAIN_PATH = r"/objective_transfer/"

In [ ]:
# importing dependencies
import matplotlib.pyplot as plt
import sklearn
import sklearn.model_selection
import skimage
import math
import numpy as np
from sklearn import preprocessing
import psutil
import tensorflow as tf
import yaml
from tensorflow.keras import layers, models
import math
from google.colab import drive
import sys
import os
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
os.chdir(MAIN_PATH+'deep_learning/CAREstd/')
# !ls
import import_ipynb
from CARE_config import *
from CARE_util import *
config = config_vars()

importing Jupyter notebook from CARE_util.ipynb


In [ ]:
# Loading preprocessed image patches
b = np.load(MAIN_PATH+'/deep_learning/npz/'+'Cervix_all_data1.npz')
# training_data = np.zeros(b['t'].shape)
training_data = b['t']
val_data = b['v']
test_data = b['te']
b = np.load(MAIN_PATH+'/deep_learning/npz/'+'Cervix_all_data_labels1.npz')
# training_data_labels = np.zeros(b['t'].shape)
training_data_labels = b['t']
val_data_labels = b['v']
test_data_labels = b['te']
del b

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

from tifffile import imread
from csbdeep.utils import axes_dict, plot_some, plot_history
from csbdeep.utils.tf import limit_gpu_memory
from csbdeep.io import load_training_data
from csbdeep.models import Config, CARE

In [ ]:
axes='YXC'
n_channel_in=1
n_channel_out=1
n_depth=2
kern_size=5
dropout= 0
n_first=32
n_channel_out=1
residual=True
last_activation='relu'
train_learning_rate = 0.0001
train_batch_size=8
train_epochs=1000

In [ ]:
train_reduce_lr = {'factor':0.99,'min_delta':0,'patience':10, 'min_lr':0.000001}
config = Config(axes, n_channel_in, n_channel_out, unet_kern_size=kern_size, train_batch_size=train_batch_size, train_steps_per_epoch=175,unet_n_depth=n_depth,train_reduce_lr=train_reduce_lr,train_learning_rate=train_learning_rate,train_epochs=train_epochs)
print(config)
vars(config)

Config(axes='YXC', n_channel_in=1, n_channel_out=1, n_dim=2, probabilistic=False, train_batch_size=8, train_checkpoint='weights_best.h5', train_checkpoint_epoch='weights_now.h5', train_checkpoint_last='weights_last.h5', train_epochs=1000, train_learning_rate=0.0001, train_loss='mae', train_reduce_lr={'factor': 0.99, 'min_delta': 0, 'patience': 10, 'min_lr': 1e-06}, train_steps_per_epoch=175, train_tensorboard=True, unet_input_shape=(None, None, 1), unet_kern_size=5, unet_last_activation='linear', unet_n_depth=2, unet_n_first=32, unet_residual=True)


{'axes': 'YXC',
 'n_channel_in': 1,
 'n_channel_out': 1,
 'n_dim': 2,
 'probabilistic': False,
 'train_batch_size': 8,
 'train_checkpoint': 'weights_best.h5',
 'train_checkpoint_epoch': 'weights_now.h5',
 'train_checkpoint_last': 'weights_last.h5',
 'train_epochs': 1000,
 'train_learning_rate': 0.0001,
 'train_loss': 'mae',
 'train_reduce_lr': {'factor': 0.99,
  'min_delta': 0,
  'min_lr': 1e-06,
  'patience': 10},
 'train_steps_per_epoch': 175,
 'train_tensorboard': True,
 'unet_input_shape': (None, None, 1),
 'unet_kern_size': 5,
 'unet_last_activation': 'linear',
 'unet_n_depth': 2,
 'unet_n_first': 32,
 'unet_residual': True}

In [ ]:
model = CARE(config, 'my_model', basedir='models')

/usr/local/lib/python3.7/dist-packages/csbdeep/models/base_model.py:149: UserWarning: output path for model already exists, files may be overwritten: /content/drive/MyDrive/Research and Grad/Diamond Program/objective_transfer/models/my_model
  warnings.warn('output path for model already exists, files may be overwritten: %s' % str(self.logdir.resolve()))


In [ ]:
model.keras_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, None, None,  0           []                               
                                 1)]                                                              
                                                                                                  
 down_level_0_no_0 (Conv2D)     (None, None, None,   832         ['input[0][0]']                  
                                32)                                                               
                                                                                                  
 down_level_0_no_1 (Conv2D)     (None, None, None,   25632       ['down_level_0_no_0[0][0]']      
                                32)                                                           

In [ ]:
print(np.shape(training_data))
a_training_data = np.zeros(np.concatenate((np.shape(training_data),[1,]),axis=0))
a_training_data_labels = np.zeros(np.concatenate((np.shape(training_data),[1,]),axis=0))
a_val_data = np.zeros(np.concatenate((np.shape(val_data),[1,]),axis=0))
a_val_data_labels = np.zeros(np.concatenate((np.shape(val_data),[1,]),axis=0))

print(np.shape(a_training_data))
a_training_data[:,:,:,0] = training_data[:,:,:]
a_training_data_labels[:,:,:,0] = training_data_labels[:,:,:]
a_val_data[:,:,:,0] = val_data[:,:,:]
a_val_data_labels[:,:,:,0] = val_data_labels[:,:,:]

(9112, 128, 128)
(9112, 128, 128, 1)


In [ ]:
print(model.config.axes)
history = model.train(a_training_data,a_training_data_labels, validation_data=(a_val_data[0:3],a_val_data_labels[0:3]))

YXC


/usr/local/lib/python3.7/dist-packages/csbdeep/models/care_standard.py:167: UserWarning: small number of validation images (only 0.0% of all images)
  warnings.warn("small number of validation images (only %.1f%% of all images)" % (100*frac_val))


Epoch 1/1000
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x7fca301a0dd0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean_or_not.<locals>.<lambda> at 0x7fca301a0dd0>: found multiple definitions with identical signatures at the location. This error may be avoided by defining each lambda on a single line and with unique argument names. The matching definitions were:
Match 0:
(lambda x: K.mean(x, axis=(- 1)))

Match 1:
(lambda x: x)

To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Cause: could not parse the source code of <function _mean

KeyboardInterrupt: ignored

In [ ]:
model.save_weights('trained_weight.h5')

In [ ]:
from datetime import datetime
now = datetime.now() # current date and time
date_time = now.strftime("%m-%d-%Y_%H-%M-%S")
plt.figure(1)
plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
#plt.ylim([0.01, 0.025])
plt.legend(loc='upper right')
plt.savefig('modelCAREstd_loss.png')
plt.figure(2)
plt.plot(history.history['mse'], label='mse')
plt.plot(history.history['val_mse'], label = 'val_mse')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
#plt.ylim([0.05, 0.11])
plt.legend(loc='upper right')
plt.figure(2)
plt.plot(history.history['mae'], label='mae')
plt.plot(history.history['val_mae'], label = 'val_mae')
plt.xlabel('Epoch')
plt.ylabel('accuracy')
#plt.ylim([0.05, 0.11])
plt.legend(loc='upper right')

In [ ]:
predictions = model.keras_model.predict(a_training_data[0:3])

In [ ]:
def revert_img(img, patch_shape, slide):
  # reverts original image and removes overlaps by splitting overlap over 2 images
  step = int(patch_shape-slide)
  reconstructed_arr = np.zeros((1024,1024))
  for x in range(img.shape[0]):
    for y in range(img.shape[1]):
      start_x = int(slide/2)
      start_y = int(slide/2)
      end_x = 0
      end_y = 0
      if x == 0:
        start_x = 0
        end_x = int(slide/2)
      if y == 0:
        start_y = 0
        end_y = int(slide/2)
      if x == img.shape[0]-1: end_x = int(slide/2)
      if y == img.shape[1]-1: end_y = int(slide/2)
      x_pos, y_pos = x * step + start_x, y * step + start_y
      reconstructed_arr[x_pos : x_pos + step + end_x, y_pos : y_pos + step + end_y] = img[x, y, start_x:start_x+step+end_x, start_y:start_y+step+end_y]
  return reconstructed_arr

def merge_patches(img, patch_shape, slide):
  #  merging patches, img is a 3D array of stacked patches
  row_len = int(math.sqrt(img.shape[0]))
  patches = np.zeros((row_len,row_len,patch_shape,patch_shape))
  print(img.shape)
  print(patches.shape)
  for r in range(row_len):
      patches[r,:,:,:] = img[r*row_len:r*row_len+row_len,:,:]
  # plt.figure(5)
  # plt.imshow(patches[1,1,:,:])
  return revert_img(patches,patch_shape, slide)
# # comparing one processed and preprocessed image
def MSE(img1, img2):
        squared_diff = (img1 -img2) ** 2
        summed = np.sum(squared_diff)
        num_pix = img1.shape[0] * img1.shape[1] #img1 and 2 should have same shape
        err = summed / num_pix
        return err
# original = merge_patches(test_data[0:25,:,:], 256, 64)
# label = merge_patches(test_data_labels[0:25,:,:], 256, 64)
# pred = merge_patches(predictions[0:25,:,:], 256, 64)
plt.figure(figsize = (10,10))
plt.hist(predictions[0,:,:,0]);
plt.title("prediction")
plt.figure(figsize = (10,10))
plt.hist(a_training_data[0,:,:,0]);
plt.title("a_training_data")
plt.figure(figsize = (10,10))
plt.hist(a_training_data_labels[0,:,:,0]);
plt.title("a_training_data_labels")
# plt.figure(figsize = (10,10))
# plt.hist(pred-original);

print("avg original", predictions[0,:,:,0].mean())
print("avg pred", a_training_data[0,:,:,0].mean())
print("avg label",a_training_data_labels[0,:,:,0].mean())
# print("avg diff", (pred-label).mean())
# print("MSE", MSE(pred,label))